First Approch

Using pretrained marathi summmarization model

In [ ]:
# Install Required Packeges
!pip install pytube transformers[sentencepiece] SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
! pip install pydub

Download youtube video

In [ ]:
# Import necessary libraries
from pytube import YouTube
import os
import moviepy.editor as mp

In [ ]:
# Define a function to download audio from a YouTube video
def download_audio(url, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get the YouTube video
    yt = YouTube(url)

    # Get the highest quality audio stream (usually in the format 'audio only')
    audio_stream = yt.streams.filter(only_audio=True).first()

    # Download the audio stream
    audio_stream.download(output_folder)

    # Get the downloaded file's path
    downloaded_file_path = os.path.join(output_folder, audio_stream.default_filename)

    # Convert the downloaded video to MP3 format
    audio_clip = mp.AudioFileClip(downloaded_file_path)
    mp3_output_path = os.path.join(output_folder, f"{yt.title}.mp3")
    audio_clip.write_audiofile(mp3_output_path)

    # Delete the original video file
    os.remove(downloaded_file_path)

    return mp3_output_path

In [ ]:
if __name__ == "__main__":
    video_url = "https://www.youtube.com/watch?v=0-GvXmJFcrw"  # Replace with the YouTube video URL
    output_folder = "downloaded_audio"  # Replace with the desired output folder

    mp3_file_path = download_audio(video_url, output_folder)
    print(f"MP3 audio downloaded and saved at: {mp3_file_path}")

MoviePy - Writing audio in downloaded_audio/महायुद्ध LIVE: अजित पवारांची एन्ट्री, महाराष्ट्रातल्या सत्ताकारणातली मोठी बिघाडी! Pawar vs Pawar.mp3


MoviePy - Done.
MP3 audio downloaded and saved at: downloaded_audio/महायुद्ध LIVE: अजित पवारांची एन्ट्री, महाराष्ट्रातल्या सत्ताकारणातली मोठी बिघाडी! Pawar vs Pawar.mp3


Convert downloaded audio file wav format and 1 minute chunk

In [ ]:
# Import necessary libraries
from pydub import AudioSegment
import os
import re

In [ ]:
# Define a function to divide audio into chunks
def divide_audio(input_file, output_folder, chunk_duration_ms):
    # Load the input MP3 audio file
    audio = AudioSegment.from_mp3(input_file)

    # Calculate the total duration of the audio in milliseconds
    total_duration_ms = len(audio)

    # Calculate the number of chunks needed
    num_chunks = total_duration_ms // chunk_duration_ms

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Divide the audio into chunks
    for i in range(num_chunks):
            start_time = i * chunk_duration_ms
            end_time = start_time + chunk_duration_ms
            chunk = audio[start_time:end_time]
            output_file = os.path.join(output_folder, f"chunk_{i+1}.wav")
            chunk.export(output_file,format="wav")



Audio divided into 1-minute chunks wav format.


In [ ]:
if __name__ == "__main__":
    input_file = "/content/downloaded_audio/महायुद्ध LIVE: अजित पवारांची एन्ट्री, महाराष्ट्रातल्या सत्ताकारणातली मोठी बिघाडी! Pawar vs Pawar.mp3"  # Replace with your input MP3 file
    output_folder = "chunk_wav_files"  # Replace with the desired output folder
    chunk_duration_ms = 60000  # 1 minute in milliseconds

    divide_audio(input_file, output_folder, chunk_duration_ms)
    print("Audio divided into 1-minute chunks wav format.")

recognize audio from chunk files using speech recognition then pass text into the summarization model and summarize the text

In [ ]:
# Import necessary libraries
import re
import speech_recognition as sr
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os

In [ ]:
# Create an empty list to store summaries
summary_list=[]

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

# Set up summarization model and tokenizer
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create a recognizer instance for speech recognition

recognizer = sr.Recognizer()

# Loop through chunk files and perform speech recognition and summarization
for i in range(115):
  i=str(i)
  for j in os.listdir("/content/chunk_wav_files"):
      k=(j.split("_")[1]).split(".")[0]
      if i == k:
        with sr.AudioFile("/content/chunk_wav_files/"+str(j)) as source:
            audio = recognizer.record(source)  # Record the entire audio file
        try:
            # Recognize the speech using Google Web Speech API
            text = recognizer.recognize_google(audio,language="mr-IN")
            # Perform text summarization
            article_text = text
            input_ids = tokenizer(
                [WHITESPACE_HANDLER(article_text)],
                return_tensors="pt",
                padding="max_length",
                truncation=True,
                max_length=1000
            )["input_ids"]

            output_ids = model.generate(
                input_ids=input_ids,
                max_length=84,
                no_repeat_ngram_size=2,
                num_beams=4
            )[0]

            summary = tokenizer.decode(
                output_ids,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False
            )
            summary_list.append(summary)
        except sr.UnknownValueError:
            print("Google Web Speech API could not understand audio")
        except sr.RequestError as e:
            print("Could not request results from Google Web Speech API; {0}".format(e))

In [ ]:
# Combine all summaries into a single text
summarize_text="".join(summary_list)

In [ ]:
# Print the summarized text
summarize_text

"अजित पवारांनी उपमुख्यमंत्रीपदाची शपथ घेऊन आज आठवण वा दिवस उजाडलेला आहे.राष्ट्रवादी काँग्रेसचे अध्यक्ष शरद पवार आणि विरोधी पक्षनेते अजित पाटील यांच्यात महाराष्ट्रात लढाई सुरू आहे.राष्ट्रवादी काँग्रेसचे अध्यक्ष शरद पवार आणि भाजपचे नेते सुप्रिया सुळे यांच्यात झालेल्या वादग्रस्त प्रयोगात आपल्या सोबत उमेश पाटील आहेतमहाराष्ट्र विधानसभा निवडणुकीच्या निकाल लागल्यानंतर भाजप आणि भारतीय जनता पक्ष या दोन्ही पक्षांच्या नेत्यांच्या गटाला घेऊन सत्तेत गेले आहेत.राष्ट्रवादी काँग्रेसचे अध्यक्ष अजितदाद अब्दुल कलाम यांनी भारतीय जनता पार्टी आणि शिवसेनेच्या एकत्रित सरकार बरोबर जाण्याचा निर्णय घेतला आहे.महाराष्ट्र विधानसभा निवडणुकीचा निर्णय एकदा नव्हे दोन-तीन वेळा फिरायला लागला आणि पुन्हा मात्र तो बदलला गेला.राष्ट्रवादी काँग्रेसचे अध्यक्ष शरद पवार यांच्या निवृत्तीची घोषणा करण्यात आली आणि त्यावेळी त्यांच्या निवडणुकीत अजित दाद यांची भूमिका घेतली गेली.पहाटेचा शपथ विधी म्हणजे अजित पवारांनी महाराष्ट्राच्या नजरेमध्ये खलनायक ठरवण्याचा प्रयत्न केला तर मग त्या बाबतीतला खुलासा का करण्यात आला?अजित पवार यांची बीजेपी अध

Second Approch

Using facebook bart model for summarization
To use this model we need english text so i convert text into english language and then suumarize text and again i convert summarize english text into the marathi text.

In [ ]:
# Install required package
! pip install googletrans==4.0.0rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=3e387cd66171bd58c62ad14db7cb2a3e97493887bdf2930c0e6af2ae85ed3e5b
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling cha

In [ ]:
# Import necessary libraries and use facebbok bart model for summarization
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


In [ ]:
from googletrans import Translator

In [ ]:
#Create empty list
text_list=[]
summary_list=[]

# Create a recognizer instance for speech recognition
recognizer = sr.Recognizer()

# Create a translator instance
translator = Translator()

# Loop through lines and process audio for summarization
for lines in range(0,120,5):
  for i in range(lines,lines+5):
    i=str(i)
    for j in os.listdir("/content/chunk_wav_files"):
        k=(j.split("_")[1]).split(".")[0]
        if i == k:
          with sr.AudioFile("/content/chunk_wav_files/"+str(j)) as source:
              audio = recognizer.record(source)  # Record the entire audio file
          try:
              # Recognize the speech using Google Web Speech API
              text = recognizer.recognize_google(audio,language="mr-IN")

              # Translate the text to English
              translated = translator.translate(text,src='mr', dest='en')
              text_list.append(translated.text)
          except Exception as e:
              print(e)
  # Combine all English text into a single string
  english_text="".join(text_list)
  # Summarize the English text
  summarize_text=summarizer(english_text, max_length=130, min_length=30, do_sample=False)

  # Translate the summarized English text back to Marathi
  translated = translator.translate(summarize_text[0]['summary_text'],src='en', dest='mr')
  summary_list.append(translated.text)
  print("***********")
  text_list.clear()

# Combine all Marathi summaries into a single text
marathi_text="".join(summary_list)
print("done")

***********
***********
***********
***********
***********
***********
***********
***********
***********
***********
***********
***********
***********
***********
***********
***********

***********
***********
***********
***********
***********
***********


Your max_length is set to 130, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


***********
***********
done


In [ ]:
# Print the Marathi summarized text
marathi_text

'आशिष जाधव: अजित पवार यांनी दिवसाची शपथ किंवा दिवस घेतला आहे.अद्याप खाते वाटपाचा कोणताही पत्ता नाही.काही एनसीपी मंत्री एनसीपीच्या कोट्यावर परत येतील, परंतु चर्चा चालू आहे.पवार साहेब यांच्या पक्षाच्या नेतृत्वात हा निर्णय तीन वेळा निर्णय घेण्यापूर्वी आणि नंतर तीन वेळा पक्षात घेण्यात आला आहे.जर आपण हे पुन्हा पुन्हा पुन्हा ठरविले तर अजित पवार महाराष्ट्रातील लोकांच्या दृष्टीने न्याय्य ठरले आहेत.सकाळी सहा वाजल्यापासून दुपारी 12:00 ते दुपारी 12 वाजता अजित वडिलांना इतरांना न विचारता इतरांना दिले जात नाही.आम्हाला हे मान्य करावेच लागेल की आमदारांनी काहीतरी दिले तेव्हा अजित दादाला काहीतरी दिले गेले होते.शरद पवारसमोर भूषण राऊतचा एक पर्याय आहे.पक्षाने त्यांना कधीही कमी पडू दिले नाही.निर्णयामध्ये कोणताही महत्त्वाचा निर्णय -वडिलांच्या संमतीशिवाय घेण्यात आला नाही.जर महाविकांनी एकत्र लढाई केली असती तर राज्याचे सर्वोच्च पद वडिलांना देण्यात आले.अजित दादाने शपथविधी घेतल्यानंतर अजित वडिलांना महाराष्ट्रात खलनायक घोषित करण्यात आले.मोहोल तालुकामध्ये राजन पाटीलचे कार्यकर्ते आणि त्यांच्या कवींनी अजित पवारचा निष

In [ ]:
len(marathi_text)

4744

In [ ]:
# Write the Marathi summarized text to a file
with open("Marathi_summary.txt",'w') as file:
  file.write(marathi_text)
  file.close()